# Modelling Libraries

In [1]:
# Import to be able to import python package from src
import sys

sys.path.insert(0, '../src')

In [2]:
import pandas as pd
import numpy as np
import ontime as on

C:\Users\vince\AppData\Local\pypoetry\Cache\virtualenvs\ontime-FclhqCOu-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---
## Generation of random time series

In [3]:
start = pd.Timestamp('2022-01-01')
end = pd.Timestamp('2022-12-31')

ts = on.generators.random_walk().generate(start=start, end=end)
ts = ts.astype(np.float32)

---
## Models

- [x] Darts
- [x] Scikit-learn API compatible regressor
- [ ] GluonTS
- [ ] Kats
- [ ] Custom PyTorch
- [ ] Custom TensorFlow

## Darts models

In [4]:
from darts.models import BlockRNNModel

In [5]:
model = on.Model(BlockRNNModel,
                 input_chunk_length=12,
                 output_chunk_length=6,
                 n_rnn_layers=2,
                 n_epochs=50
                 )
model.fit(ts)
model.predict(5)

darts.models.forecasting.torch_forecasting_model INFO  Train dataset contains 348 samples.
darts.models.forecasting.torch_forecasting_model INFO  Time series values are 32-bits; casting model to float32.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | rnn           | RNN              | 2.0 K 
4 | fc            | Sequential       | 156   
---------------------------------------------------
2.2 K     Trainable params
0         Non-trainable params
2.2 K     Total params
0.009     Total estimated model params size (MB)


Epoch 49: 100%|██████████| 11/11 [00:00<00:00, 91.02it/s, train_loss=4.360] 

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 11/11 [00:00<00:00, 91.02it/s, train_loss=4.360]

GPU available: False, used: False


TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 133.20it/s]


<TimeSeries (DataArray) (time: 5, component: 1, sample: 1)>
array([[[-8.874834]],

       [[-8.592285]],

       [[-8.365063]],

       [[-8.642714]],

       [[-8.388067]]], dtype=float32)
Coordinates:
  * time       (time) datetime64[ns] 2023-01-01 2023-01-02 ... 2023-01-05
  * component  (component) object 'random_walk'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

## Scikit-learn API compatible models

In [6]:
from sklearn.neural_network import MLPRegressor

In [7]:
model = on.Model(MLPRegressor,
                 lags=30)
model.fit(ts)
model.predict(5)

C:\Users\vince\AppData\Local\pypoetry\Cache\virtualenvs\ontime-FclhqCOu-py3.11\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


<TimeSeries (DataArray) (time: 5, component: 1, sample: 1)>
array([[[-14.11805517]],

       [[-13.86775146]],

       [[-13.44756423]],

       [[-13.60795347]],

       [[-14.09233191]]])
Coordinates:
  * time       (time) datetime64[ns] 2023-01-01 2023-01-02 ... 2023-01-05
  * component  (component) object 'pred'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

## Multi series models skforecast

In [10]:
model = on.Model(MLPRegressor(),
                 multi_series=True,
                 lags=30)

ts2 = on.generators.random_walk().generate(start=start, end=end)
ts2 = ts2.astype(np.float32)

## Add value from ts2 to ts in a new column
df = ts.pd_dataframe()
df['ts2'] = ts2.pd_dataframe()

end_train = '2022-09-23'
data_train = df.loc[:end_train, :].copy()
data_test = df.loc[end_train:, :].copy()

model.fit(ts=data_train)
model.predict(n=2)

C:\Users\vince\AppData\Local\pypoetry\Cache\virtualenvs\ontime-FclhqCOu-py3.11\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


<TimeSeries (DataArray) (time: 2, component: 2, sample: 1)>
array([[[6.13584309],
        [1.38839066]],

       [[6.28192004],
        [1.70971506]]])
Coordinates:
  * time       (time) datetime64[ns] 2022-09-24 2022-09-25
  * component  (component) object 'random_walk' 'ts2'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None